# Leer y escribir audio en Python

**Curso**: CC5213 - Recuperación de Información Multimedia  
**Profesor**: Juan Manuel Barrios  
**Fecha**: 28 de septiembre de 2023

## Decodificando el Audio

Un audio decodificado es un array de números, que representan el muestreo de la amplitud de la señal acústica a una tasa de tiempo fija.

Para leer un archivo de audio se puede usar una herramienta para convertir un audio desde su formato comprimido (como un .mp3) a un formato descomprimido (como un array de bytes). El software más usado para codificar y decodificar audio y video es **FFmpeg** https://ffmpeg.org/

Para instalarlo, en Linux basta hacer `apt install ffmpeg`. En Windows entrar a [Downloads](https://www.gyan.dev/ffmpeg/builds/), descargar `ffmpeg-release-full.7z` y obtener el archivo `ffmpeg.exe`.

Al decodificar un audio es necesario decidir dos atributos:

 * **Tasa de muestreo** (sample rate). Cantidad de muestras por segundo. Comúnmente se usa 44100 samples/segundo para alta calidad. Se puede bajar a 22050, 11025, o incluso a 8192 para voz.
 
 * **Profundidad** (bit depth). Señala la cantidad de valores posibles de cada sample (la resolución). Comúnmente se usa 16 bits con signo `s16le`, es decir, cada sample es un número entero entre -32.768 y 32.767. Para edición de audio se suele usar 32 bits (cada sample es un float).
 
 
### Crear y leer archivos .raw:

Para decodificar un audio y obtener los samples en formato crudo ("`.raw`") usar el siguiente comando (Ver https://trac.ffmpeg.org/wiki/audio%20types):

```
ffmpeg -i video.mp4 -ac 1 -ar 8192 -acodec pcm_s16le -f s16le audio.raw
```

 * `-i video.mp4` dice que se deber leer video.mp4 como entrada (input).    
 * `-ac 1` dice que se desea crear solo un canal de salida (mono).  
 * `-ar 8192` dice que la salida debe tener 8k samples por segundo.   
 * `-acodec pcm_s16le` dice que la salida debe tener un audio pcm_s16le (codificación).    
 * `-f s16le` dice que el archivo de salida (container) deben ser los bytes directamente (este parámetro es redundante ya que es el formato por defecto para el codec pcm_s16le).
 * `audio.raw` es el nombre del archivo de salida.

Este comando crea el archivo `audio.raw` con los bytes de los samples con profundidad `s16le` y con una tasa de sampleo de 8192. El archivo `audio.raw` se pueden leer con python cargando los bytes del archivo a un array de enteros 16 bits con:
```
samples = numpy.fromfile("audio.raw", dtype=numpy.int16)
```

Para reproducir los samples en memoria se puede usar un mini-reproductor que recibe el array de samples y la tasa de sampleo:
```
    import IPython.display as ipd    
    ipd.Audio(samples, rate=8192)
```

Como el audio es un array de números, se puede operar con ellos normalmente (multiplicarlos, sumarlos, etc.). Solo se debe tener cuidado de no superar el límite numérico de la profundidad usada (-32.768 y 32.767 para 16 bits).

Para volver a guardarlos en un archivo .raw hacer:
```
samples.tofile("audio.raw")
```


### Formato .raw y .wav:

El formato `.raw` solo contiene los bytes de los samples, por lo que es necesario guardar en alguna parte la tasa de sampleo (sample_rate) y la profundidad (bit depth) para poder interpretarlos bien. La ventaja es que no requiere de ninguna librería especial para su lectura.

Por otra parte, el formato `.wav` es esencialmente el mismo formato `.raw` pero que incluye un encabezado donde guarda el sample_rate y el bit depth usado en el archivo. Para poder leer un archivo `.wav` es necesario usar una librería que soporte el formato.

### Reproducir un archivo .raw:

Para reproducir los samples guardados en un archivo `audio.raw` se puede usar `ffplay` o `vlc` (https://www.videolan.org/vlc/).

 * **Reproducir archivo raw (opción 1)**: `ffplay -f s16le -acodec pcm_s16le -ar 8192 audio.raw`
 * **Reproducir archivo raw (opción 2)**: `vlc --demux=rawaud --rawaud-channels 1 --rawaud-samplerate 8192 --rawaud-fourcc=s16l audio.raw`

Es posible crear un archivo .mp3 (u otro formato) con FFmpeg:

 * **Codificar audio**: `ffmpeg -f s16le -acodec pcm_s16le -ar 8192 -i audio.raw audio.mp3`



## Leyendo Audio

Dado un video, usa ffmpeg para guardar el audio en un archivo que lee a memoria.

Para reproducir el audio se usará VLC.

In [ ]:
import numpy
import os.path
import subprocess

#funcion que recibe un nombre de archivo y llama a FFmpeg para crear un archivo raw
def extraer_audio_raw(file_video, sample_rate):
    file_raw = "{}.{}.raw".format(file_video, sample_rate) 
    if os.path.isfile(file_raw):
        return file_raw
    comando = ["ffmpeg", "-i", file_video, "-ac", "1", "-ar", str(sample_rate),
                "-acodec", "pcm_s16le", "-f", "s16le",  file_raw]
    print("INICIANDO: {}".format(" ".join(comando)))
    code = subprocess.call(comando)
    if code != 0:
        raise Exception("ERROR en comando: " + " ".join(comando))
    return file_raw

#funcion que recibe un archivo raw y llama a vlc para reproducirlo
def reproducir_audio_raw(file_raw, sample_rate):
    comando = ["vlc", "--demux=rawaud", "--rawaud-channels", "1", "--rawaud-samplerate", 
               str(sample_rate), "--rawaud-fourcc=s16l", file_raw]
    print("INICIANDO: {}".format(" ".join(comando)))
    code = subprocess.call(comando)
    if code != 0:
        raise Exception("ERROR en comando: " + " ".join(comando))


### Probando con un archivo de audio de ejemplo.

Reproducirá en VLC un audio de 1 minuto y 28 segundos.

In [ ]:
file_vivaldi = "vivaldi.mp3"
samplerate_vivaldi = 22050

file_raw_vivaldi = extraer_audio_raw(file_vivaldi, samplerate_vivaldi)
samples_vivaldi = numpy.fromfile(file_raw_vivaldi, dtype=numpy.int16);

print("  cantidad samples vivaldi = {}".format(len(samples_vivaldi)))
print("  largo = {:.1f} segundos".format( len(samples_vivaldi) / samplerate_vivaldi ))
print("  min y max sample = {} {}".format( min(samples_vivaldi), max(samples_vivaldi)))
print(samples_vivaldi)
print()
print("reproduciendo: {}".format(file_raw_vivaldi))
reproducir_audio_raw(file_raw_vivaldi, samplerate_vivaldi)

### Otro archivo de audio de ejemplo

Reproduce en VLC un audio de 30 segundos.

In [ ]:
file_jaivas = "jaivas.mp3"
samplerate_jaivas = 22050

file_raw_jaivas = extraer_audio_raw(file_jaivas, samplerate_jaivas)
samples_jaivas = numpy.fromfile(file_raw_jaivas, dtype=numpy.int16);

print("  cantidad samples jaivas = {}".format(len(samples_jaivas)))
print("  largo = {:.1f} segundos".format( len(samples_jaivas) / samplerate_jaivas ))
print("  min y max sample = {} {}".format( min(samples_jaivas), max(samples_jaivas)))
print(samples_jaivas)
print()
print("reproduciendo: {}".format(file_raw_jaivas))
reproducir_audio_raw(file_raw_jaivas, samplerate_jaivas)

### Graficando ambos audios

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

#funcion que imprime un array de samples
def mostrar_samples(samples_array, titulo):
    plt.figure(figsize=(25,5))
    plt.title(titulo, {"fontsize":32})
    plt.plot(samples_array)
    plt.xlabel('Samples')
    plt.ylabel('Amplitud')
    plt.ylim(top=32767, bottom=-32768)
    plt.yticks(numpy.arange(-30000, 30001, step=10000))
    plt.margins(0)
    plt.show()

mostrar_samples(samples_jaivas, "Vivaldi")

mostrar_samples(samples_vivaldi, "Jaivas")


### Sumar ambas ondas de audio

Ambos audios deben tener el mismo sample rate. Se debe tener cuidado de no superar el límite de los 16 bits.  
El resultado se guarda en un archivo `.raw` y se reproduce con VLC.

In [ ]:
def sumar_samples(samples1, samples2):
    largo = min(len(samples1), len(samples2))
    nuevos_samples = numpy.zeros(largo, dtype=numpy.int16) 
    for i in range(largo):
        #sample audio 1
        val1 = int(samples1[i])
        #sample audio 2
        val2 = int(samples2[i])
        # sumar ambos
        val = int((val1 + val2))
        # saturacion
        if val > 32767:
            val = 32767
        elif val < -32768:
            val = -32768
        # guardar el nuevo valor
        nuevos_samples[i] = val 
    return nuevos_samples

nuevos_samples = sumar_samples(samples_vivaldi, samples_jaivas)

nuevo_archivo_raw = "suma.raw" 

print("guardando archivo {}".format(nuevo_archivo_raw))
nuevos_samples.tofile(nuevo_archivo_raw)

mostrar_samples(nuevos_samples, "Vivaldi+Jaivas")

print("reproduciendo {}".format(nuevo_archivo_raw))
reproducir_audio_raw(nuevo_archivo_raw, samplerate_vivaldi)


### Efecto de eco (repetir el mismo audio con cierto desfase)

Se copia la misma onda con un leve desfase. Se usa el reproductor interno de python.

In [ ]:
import IPython.display as ipd    
    
def hacer_eco(samples, samplerate, segundos_desfase):
    samples_desfase = int(samplerate * segundos_desfase)
    nuevos_samples = numpy.zeros(len(samples), dtype=numpy.int16) 
    for i in range(samples_desfase, len(samples)):
        #sample actual
        val1 = int(samples[i])
        #sample anterior
        val2 = int(samples[i - samples_desfase])
        # sumar ambos, menor ponderacion al eco
        val = int((val1 + 0.5 * val2))
        # saturacion
        if val > 32767:
            val = 32767
        elif val < -32768:
            val = -32768
        # guardar el nuevo valor
        nuevos_samples[i] = val 
    return nuevos_samples

samples_eco = hacer_eco(samples_jaivas, samplerate_jaivas, 0.3)
ipd.Audio(samples_eco, rate=samplerate_jaivas, autoplay=True)
